This notebook creates csv files to be used to train ML models. It also creates csv files containing data in a format to predict future years.

In [2]:
import pandas as pd
import numpy as np

In [4]:
player_df = pd.read_csv('Data/NHL_player_season_log_all.csv')
season_df = pd.read_csv('Data/League_Average_Data.csv')

In [5]:
display(player_df.head())
season_df.head()

,Season,Age,Tm,GP,G,A,PTS,+/-,PIM,EV,...,ATOI,Player,TSA,FOW,FOL,FO%,BLK,HIT,TK,GV
0,1997-98,22.0,MDA,3,0,0,0,-1.0,0,0.0,...,NaN,Antti Aalto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998-99,23.0,MDA,73,3,5,8,-12.0,24,1.0,...,9:23,Antti Aalto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1999-00,24.0,MDA,63,7,11,18,-13.0,26,6.0,...,13:11,Antti Aalto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01,25.0,MDA,12,1,1,2,1.0,2,1.0,...,10:37,Antti Aalto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-14,25.0,TOR,1,0,0,0,-2.0,0,0.0,...,5:16,Spencer Abbott,2.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0


,year,games,total_goals_per_game,goals_pp,chances_pp,shots,shot_pct
0,1976-77,80,6.64,53,265,2401,11.1
1,1977-78,80,6.59,54,255,2343,11.3
2,1978-79,80,7.00,62,271,2351,11.9
3,1979-80,80,7.03,61,280,2352,12.0
4,1980-81,80,7.69,77,340,2441,12.6


In [11]:
player_df.columns

Index(['Season', 'Age', 'Tm', 'GP', 'G', 'A', 'PTS', '+/-', 'PIM', 'EV', 'PP',
       'SH', 'GW', 'EV.1', 'PP.1', 'SH.1', 'S', 'S%', 'TOI', 'ATOI', 'Player',
       'TSA', 'FOW', 'FOL', 'FO%', 'BLK', 'HIT', 'TK', 'GV'],
      dtype='object')

In [117]:
player_df[player_df.Player=='Pontus Aberg']

,Season,Age,Tm,GP,G,A,PTS,+/-,PIM,EV,...,ATOI,Player,TSA,FOW,FOL,FO%,BLK,HIT,TK,GV
19,2016-17,23.0,NSH,15,1,1,2,-2.0,4,1.0,...,12:20,Pontus Aberg,24.0,1.0,0.0,100.0,2.0,9.0,5.0,2.0
20,2017-18,24.0,TOT,53,4,12,16,9.0,10,4.0,...,12:10,Pontus Aberg,125.0,4.0,8.0,33.3,8.0,24.0,12.0,20.0
21,2017-18,24.0,NSH,37,2,6,8,8.0,8,2.0,...,11:06,Pontus Aberg,69.0,4.0,6.0,40.0,7.0,16.0,11.0,7.0
22,2017-18,24.0,EDM,16,2,6,8,1.0,2,2.0,...,14:38,Pontus Aberg,56.0,0.0,2.0,0.0,1.0,8.0,1.0,13.0
23,2018-19,25.0,TOT,59,12,13,25,-14.0,20,9.0,...,14:36,Pontus Aberg,211.0,2.0,17.0,10.5,11.0,45.0,15.0,36.0
24,2018-19,25.0,ANA,37,11,8,19,-10.0,14,8.0,...,15:37,Pontus Aberg,153.0,2.0,9.0,18.2,7.0,31.0,13.0,31.0
25,2018-19,25.0,MIN,22,1,5,6,-4.0,6,1.0,...,12:52,Pontus Aberg,58.0,0.0,8.0,0.0,4.0,14.0,2.0,5.0
26,2019-20,26.0,TOR,5,0,1,1,0.0,0,0.0,...,8:42,Pontus Aberg,5.0,0.0,0.0,NaN,1.0,1.0,1.0,2.0


In [20]:
# For seasons where a player was traded mid-season, there are multiple row entries. This removes the repeated information.
for player in player_df[player_df.Tm=='TOT'].Player.unique():
    season_list = player_df[(player_df.Player == player) & (player_df.Tm=='TOT')].Season.unique()
    
    for season in season_list:
        index = player_df.index[(player_df.Player == player) & (player_df.Tm !='TOT') & (player_df.Season == season)].tolist()
        player_df.drop(index=index,inplace=True)


In [21]:
# Remove columns that aren't currently being used in models
player_df.drop(columns=['Tm','+/-', 'PIM', 'EV', 'PP','SH', 'GW', 'EV.1', 'PP.1', 'SH.1','ATOI','TSA', 'FOW', 'FOL', 'FO%', 'BLK', 'HIT', 'TK', 'GV'],inplace=True)
player_df

,Season,Age,GP,G,A,PTS,S,S%,TOI,Player
0,1997-98,22.0,3,0,0,0,1.0,0.0,NaN,Antti Aalto
1,1998-99,23.0,73,3,5,8,61.0,4.9,685.0,Antti Aalto
2,1999-00,24.0,63,7,11,18,102.0,6.9,830.0,Antti Aalto
3,2000-01,25.0,12,1,1,2,18.0,5.6,127.0,Antti Aalto
4,2013-14,25.0,1,0,0,0,2.0,0.0,5.0,Spencer Abbott
...,...,...,...,...,...,...,...,...,...,...
34747,2002-03,25.0,67,4,13,17,113.0,3.5,1448.0,Andrei Zyuzin
34750,2003-04,26.0,65,8,13,21,104.0,7.7,1324.0,Andrei Zyuzin
34751,2005-06,28.0,57,7,11,18,80.0,8.8,1076.0,Andrei Zyuzin
34752,2006-07,29.0,49,1,5,6,36.0,2.8,677.0,Andrei Zyuzin


In [22]:
# Create dict that holds player season numbers
player_dict = dict()
for player in player_df.Player.unique():
    print(f'\r{player}                           ',end='')
        
    player_dict[player] = dict()
    for season_num,season in enumerate(player_df[player_df.Player == player].Season.unique()):
        player_dict[player][season] = season_num+1


Andrei Zyuzin                                           

In [23]:
def player_season(row):
    global player_dict
    return player_dict[row.Player][row.Season]

In [24]:
player_df['Player_Season_Num'] = player_df.apply(player_season, axis = 1)

In [31]:
# Checking that the above worked as intended
display(player_df[player_df.Player=='Joe Thornton'])

,Season,Age,GP,G,A,PTS,S,S%,TOI,Player,Player_Season_Num
31440,1997-98,18.0,55,3,4,7,33.0,9.1,NaN,Joe Thornton,1
31441,1998-99,19.0,81,16,25,41,128.0,12.5,1243.0,Joe Thornton,2
31442,1999-00,20.0,81,23,37,60,171.0,13.5,1725.0,Joe Thornton,3
31443,2000-01,21.0,72,37,34,71,181.0,20.4,1566.0,Joe Thornton,4
31444,2001-02,22.0,66,22,46,68,152.0,14.5,1319.0,Joe Thornton,5
31445,2002-03,23.0,77,36,65,101,196.0,18.4,1736.0,Joe Thornton,6
31446,2003-04,24.0,77,23,50,73,187.0,12.3,1666.0,Joe Thornton,7
31447,2005-06,26.0,81,29,96,125,195.0,14.9,1728.0,Joe Thornton,8
31448,2005-06,26.0,23,9,24,33,60.0,15.0,496.0,Joe Thornton,8
31449,2005-06,26.0,58,20,72,92,135.0,14.8,1233.0,Joe Thornton,8


In [9]:
# Merge player and league season dataframes
df = pd.merge(left=player_df, right=season_df, left_on='Season', right_on='year')

In [10]:
df.sort_values(by=['Player','Player_Season_Num'],inplace=True)
df.reset_index()

,index,Season,Age,GP,G,A,PTS,S,S%,TOI,Player,Player_Season_Num,year,games,total_goals_per_game,goals_pp,chances_pp,shots,shot_pct
0,4474,2016-17,20.0,5,0,1,1,8.0,0.0,68.0,A.J. Greer,1,2016-17,82,5.45,47,245,2478,9.0
1,12406,2017-18,21.0,17,0,3,3,13.0,0.0,126.0,A.J. Greer,2,2017-18,82,5.86,50,250,2620,9.2
2,13299,2018-19,22.0,15,1,1,2,9.0,11.1,94.0,A.J. Greer,3,2018-19,82,5.96,47,239,2579,9.5
3,18328,1980-81,20.0,2,0,0,0,1.0,0.0,NaN,Aaron Broten,1,1980-81,80,7.69,77,340,2441,12.6
4,18770,1981-82,21.0,58,15,24,39,67.0,22.4,NaN,Aaron Broten,2,1981-82,80,8.03,73,320,2489,12.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29237,2998,2000-01,28.0,73,38,51,89,217.0,17.5,1443.0,Ziggy Palffy,8,2000-01,82,5.51,63,376,2272,9.9
29238,26664,2001-02,29.0,63,32,27,59,161.0,19.9,1279.0,Ziggy Palffy,9,2001-02,82,5.24,53,338,2262,9.5
29239,15359,2002-03,30.0,76,37,48,85,277.0,13.4,1706.0,Ziggy Palffy,10,2002-03,82,5.31,60,363,2326,9.4
29240,16249,2003-04,31.0,35,16,25,41,109.0,14.7,754.0,Ziggy Palffy,11,2003-04,82,5.14,57,348,2300,9.2


In [144]:
# Create feature df, including labels for each row

current_year_stat_list = [
    #Year Stats
    'Age','Player_Season_Num','PTS','GP'
]

prev_year_stat_list = [
    #Previous Year Stats
    'GP','G','A','PTS','S','TOI'
]

league_stat_list = [
    #Previous Year League Stats
    'games','total_goals_per_game','goals_pp','chances_pp','shots','shot_pct'
]

career_stat_list = [
    #Career stats up until year
    'GP','G','A','PTS','S'
]

# Want to predict 1-3 years into the future
for years_back in range(1,4):
    print(f'Years Back: {years_back}')
    player_stat_dict = {
        'Player': [],
        #Year Stats
        'Age' : [],
        'Player_Season_Num' : [],
        'PTS' : [],
        'GP' : [],
        #Prev Year Stats
        'prev_GP' : [],
        'prev_G' : [],
        'prev_A' : [],
        'prev_PTS' : [],
        'prev_S' : [],
        'prev_TOI' : [],

        #Career Stats
        'career_GP' : [],
        'career_G' : [],
        'career_A' : [],
        'career_PTS' : [],
        'career_S' : [],

        #League Stats
        'games': [],
        'total_goals_per_game': [],
        'goals_pp': [],
        'chances_pp': [],
        'shots': [],
        'shot_pct': []
    }


    for player in df.Player.unique():
        tmp_df = df[df.Player==player]
        if len(tmp_df)<years_back+1:
            continue
        print(f'\r{player}                           ',end='')
        for n in range(years_back,len(tmp_df)):
            player_stat_dict['Player'].append(player)
            for stat in current_year_stat_list:
                player_stat_dict[stat].append(tmp_df[stat].iloc[n])

            for stat in prev_year_stat_list:
                player_stat_dict[f'prev_{stat}'].append(tmp_df[stat].iloc[n-years_back])

            for stat in league_stat_list:
                player_stat_dict[stat].append(tmp_df[stat].iloc[n-years_back])

            for stat in career_stat_list:
                player_stat_dict[f'career_{stat}'].append(tmp_df[stat].iloc[0:n+1-years_back].sum())


    print(f'\rCompleted               ')

    feature_df = pd.DataFrame(player_stat_dict)

    feature_df.to_csv(f'player_data_{years_back}_year{"s" if years_back>1 else ""}_back.csv', index=False)

Years Back: 1
Completed                                               
Years Back: 2
Completed                                               
Years Back: 3
Completed                                               


In [25]:
# Creating databases to use in predictions, similar to above but looking forward instead of back

current_year_stat_list = [
    #Year Stats
    'PTS','GP'
]

prev_year_stat_list = [
    #Previous Year Stats
    'GP','G','A','PTS','S','TOI'
]

league_stat_list = [
    #Previous Year League Stats
    'games','total_goals_per_game','goals_pp','chances_pp','shots','shot_pct'
]

career_stat_list = [
    #Career stats up until year
    'GP','G','A','PTS','S'
]


for years_back in range(1,4):
    print(f'Years Back: {years_back}')
    player_stat_dict = {
        'Player': [],
        'Season': [],
        #Year Stats
        'Age' : [],
        'Player_Season_Num' : [],
        'PTS' : [],
        'GP' : [],
        #Prev Year Stats
        'prev_GP' : [],
        'prev_G' : [],
        'prev_A' : [],
        'prev_PTS' : [],
        'prev_S' : [],
        'prev_TOI' : [],

        #Career Stats
        'career_GP' : [],
        'career_G' : [],
        'career_A' : [],
        'career_PTS' : [],
        'career_S' : [],

        #League Stats
        'games': [],
        'total_goals_per_game': [],
        'goals_pp': [],
        'chances_pp': [],
        'shots': [],
        'shot_pct': []
    }


    for player in df.Player.unique():
        tmp_df = df[df.Player==player]
        print(f'\r{player}                           ',end='')
        for n in range(len(tmp_df)):
            player_stat_dict['Player'].append(player)
            player_stat_dict['Age'].append(tmp_df['Age'].iloc[n]+years_back)
            player_stat_dict['Player_Season_Num'].append(tmp_df['Player_Season_Num'].iloc[n]+years_back)
            player_stat_dict['Season'].append(f'{str(int(tmp_df.Season.iloc[n][:4])+years_back)}-{str(int(tmp_df.Season.iloc[n][:4])+years_back+1)[-2:]}')
            for stat in current_year_stat_list:
                player_stat_dict[stat].append(tmp_df[stat].iloc[n])

            for stat in prev_year_stat_list:
                player_stat_dict[f'prev_{stat}'].append(tmp_df[stat].iloc[n])

            for stat in league_stat_list:
                player_stat_dict[stat].append(tmp_df[stat].iloc[n])

            for stat in career_stat_list:
                player_stat_dict[f'career_{stat}'].append(tmp_df[stat].iloc[0:n+1].sum())


    print(f'\rCompleted               ')

    feature_df = pd.DataFrame(player_stat_dict)

    feature_df.to_csv(f'player_prediction_data_no_labels_{years_back}_year{"s" if years_back>1 else ""}_back.csv', index=False)

Years Back: 1
Completed                                               
Years Back: 2
Completed                                               
Years Back: 3
Completed                                               
